# Compare momentum and spin angles
This is from an MDC2 MC production. 

We'll use `Heist.jl` which is a Julia wrapper around gallery that Adam wrote. See https://github.com/lyon-fnal/Heist.jl. 

In [1]:
using Cxx, ROOT, Heist, Plots, ProgressMeter

## Find data

Let's find some data in `/pnfs`. 

In [2]:
const filesDir = "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784"
files = readlines(`find $filesDir -name 'gm2*.30?*'`)  # Pick a subset of the files
println("There are $(length(files)) files")

There are 11 files


Let's look at one of the files to see what data products are in it. We can use the `product_sizes_dumper` command from _art_. 

In [3]:
files[1]

"/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.302.root"

In [4]:
;product_sizes_dumper $(files[1]) \| grep ::

         723258918      0.661 gm2truth::TrajectoryArtRecords_artg4__mdc2.
         244324304      0.223 gm2truth::TrackingActionArtRecords_artg4__mdc2.
          63004078      0.058 gm2truth::RingArtRecords_artg4_Ring_mdc2.
          37009378      0.034 gm2truth::LookupArtRecords_artg4_calorimeter_mdc2.
           7901880      0.007 gm2truth::RingTrackingPlaneArtRecords_artg4_RingTrackingPlanes_mdc2.
           6670241      0.006 gm2truth::GhostDetectorArtRecords_artg4_trackerdummyplane_mdc2.
           3557883      0.003 gm2truth::GhostFiberHarpArtRecords_artg4_ghostFiberHarp_mdc2.
           2155258      0.002 gm2truth::IBMSTruthArtRecords_artg4_ibms_mdc2.
           1792840      0.002 art::RNGsnapshots_randomsaver__mdc2.
           1110349      0.001 gm2truth::XtalArtRecords_artg4_calorimeter_mdc2.
           1077636      0.001 gm2truth::T0ArtRecords_artg4_t0_mdc2.
            772505      0.001 gm2truth::IBMSFiberArtRecords_artg4_ibms_mdc2.
            524512      0.000 gm2truth::Ph

Looks like we want the `RingTrackingPlaneArtRecords` with module label of `artg4` and instance label of `RingTrackingPlanes`. Here are details:

* Data product definition at [RingTrackingPlanesArtRecord.hh](https://redmine.fnal.gov/redmine/projects/gm2dataproducts/repository/revisions/v7_06_05/entry/mc/ring/RingTrackingPlaneArtRecord.hh) ... note it refers to [BasicArtRecord.hh](https://redmine.fnal.gov/redmine/projects/gm2dataproducts/repository/revisions/v7_06_05/entry/mc/basic/BasicArtRecord.hh)
* It gets made with [RingTrackingPlanes_service.hh](https://redmine.fnal.gov/redmine/projects/gm2ringsim/repository/revisions/v7_06_05/entry/ring/RingTrackingPlanes_service.hh) and [RingTrackingPlanes_service.cc](https://redmine.fnal.gov/redmine/projects/gm2ringsim/repository/revisions/v7_06_05/entry/ring/RingTrackingPlanes_service.cc)

We can look at the FHICL parameters with (note the `grep -A ...` means to show more lines after the match)

In [5]:
;config_dumper --services $(files[1]) \| grep  -A 4 RingTrackingPlanes

RingTrackingPlanes: {
   nplanes: 4
   phi0: -1
   verbosity: 0
}


Relevant parts of the construction code are ...

```cxx
phi0_{pset.get<double>("phi0", 0) * deg},
```

and

```cxx
for (int nplane=0; nplane!=nplanes_; ++nplane)
{
   // ...
   // Calculate downstream angle from 12 o'clock
   // in global gm2ringsim coordinates
   double angle = phi0_ + CLHEP::twopi * nplane / nplanes_;
   auto rot = new G4RotationMatrix();
   rot->rotateZ(angle);
   // ...
 ```
 
 Not sure why we start at -1 degree. 
 
 Also, note that 
 ```cxx
 typedef std::vector<RingTrackingPlaneArtRecord> RingTrackingPlaneArtRecordCollection;
 ```

## Setup Heist & Gallery

In [6]:
mdc2_1033 = HeistFiles(files) ; # The trailing semi-colon prevents a long printoutd

Define the data we want to extract

In [7]:
rtpRecordSpec = HeistRecordSpec(
                    namespace     = "gm2truth",
                    productType   = "RingTrackingPlaneArtRecordCollection",
                    header        = "gm2dataproducts/mc/ring/RingTrackingPlaneArtRecord.hh",
                    moduleLabel   = "artg4", instanceLabel = "RingTrackingPlanes")

Heist.HeistRecordSpec("gm2truth::RingTrackingPlaneArtRecordCollection", (class art::InputTag) {
}
)

## Practice on one event
Let's play with one event to see how things look. We need to find an event with a few RingTrackingPlane Hits

In [8]:
he = HeistEvent(mdc2_1033)

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.302.root


Heist.HeistEvent((class gallery::Event) {
}
, 1)

In [9]:
nextEvent(he)

Heist.HeistEvent((class gallery::Event) {
}
, 2)

In [10]:
rtp = getRecord(he, rtpRecordSpec)

(class gallery::ValidHandle<class std::vector<struct gm2truth::RingTrackingPlaneArtRecord, class std::allocator<struct gm2truth::RingTrackingPlaneArtRecord> > >) {
}


In [11]:
icxx"$rtp->size();"

0x0000000000000001

There's only one tracking plane hit

In [12]:
rtp

(class gallery::ValidHandle<class std::vector<struct gm2truth::RingTrackingPlaneArtRecord, class std::allocator<struct gm2truth::RingTrackingPlaneArtRecord> > >) {
}


In [14]:
rtpv = icxx"""(*$rtp);"""

(class std::vector<struct gm2truth::RingTrackingPlaneArtRecord, class std::allocator<struct gm2truth::RingTrackingPlaneArtRecord> > &) {
}


In [15]:
@cxx rtpv->size()

0x0000000000000001

In [20]:
icxx"""$rtpv[0];"""

:3:35: error: expected unqualified-id
            auto &r = __juliavar1.;
                                  ^


ErrorException: [91mTried to Emit Invalid Decl[39m

In [30]:
icxx"""return $(rtpRecordSpec.artInputTag).encode();"""

:3:35: error: expected unqualified-id
            auto &r = __juliavar1.;
                                  ^


ErrorException: [91mTried to Emit Invalid Decl[39m

In [24]:
icxx"""return $(he.galleryEvent).getValidHandle<gm2truth::RingTrackingPlaneArtRecord>($(rtpRecordSpec.artInputTag));"""

LoadError: [91mUnrecognized C++ Exception (N3cet15coded_exceptionIN3art6errors10ErrorCodesEXadL_ZNS1_15ExceptionDetail9translateB5cxx11ES3_EEEE)[39m

In [40]:
function findEventWithManyRTPHits(heistFiles, maxEvents)
    p = Progress(maxEvents)
    local rtpHits
    for anEvent in HeistEvents(heistFiles, maxEvents)
        rtpHits = getRecord(anEvent, rtpRecordSpec)
        if icxx"$rtpHits->size()" > 0
            break
        end
    end
    
    return rtpHits
end

findEventWithManyRTPHits(heistFilesPnfs, 200)

findEventWithManyRTPHits (generic function with 1 method)

In [ ]:
it = "artg4:RingTrackingPlanes"
icxx"""art::InputTag it("artg4:); return it;"""

In [1]:
using PyCall

INFO: Precompiling module PyCall.


In [2]:
@pyimport ROOT

In [8]:
function read_header(h)
    ROOT.gROOT[:ProcessLine]("""#include "$h" """)
end

read_header (generic function with 1 method)

In [9]:
read_header("gallery/ValidHandle.h")

0

In [11]:
function provide_get_valid_handle(klass)
    ROOT.gROOT[:ProcessLine]("""
       template gallery::ValidHandle<$klass> gallery::Event::getValidHandle<$klass>(art::InputTag const&) const;
         """)
end

provide_get_valid_handle (generic function with 1 method)

In [12]:
provide_get_valid_handle("std::vector<gm2truth::RingTrackingPlaneArtRecord>")

0

In [13]:
tag = ROOT.art.InputTag("artg4:RingTrackingPlanes")

LoadError: [91mUndefVarError: art not defined[39m

In [14]:
ROOT[:art]

LoadError: [91mMethodError: no method matching getindex(::Module, ::Symbol)[39m

In [15]:
ROOT.art

LoadError: [91mUndefVarError: art not defined[39m

In [17]:
@pyimport ROOT.art as art
@pyimport ROOT.gallery as gallery
@pyimport ROOT.vector as rvector

In [18]:
gallery

__anon__

In [19]:
art

__anon__

In [20]:
art[:InputTag]

LoadError: [91mMethodError: no method matching getindex(::Module, ::Symbol)[39m

In [21]:
art.InputTag

LoadError: [91mUndefVarError: InputTag not defined[39m

In [22]:
@pyimport ROOT

In [23]:
ROOT.art

LoadError: [91mUndefVarError: art not defined[39m

In [24]:
py"""import ROOT"""

LoadError: [91mPyError (ccall(@pysym(:Py_CompileString), PyPtr, (Cstring, Cstring, Cint), sb, fname, input_type)) <type 'exceptions.SyntaxError'>
SyntaxError('invalid syntax', ('/gm2/app/users/lyon/julia/pkgs/v0.6/PyCall/src/pyeval.jl', 1, 6, 'import ROOT'))
[39m

In [26]:
r = pyimport("ROOT")

PyObject <module 'ROOT' from '/cvmfs/gm2.opensciencegrid.org/prod8/external/root/v6_08_06g/Linux64bit+2.6-2.12-e14-prof/lib/ROOT.pyc'>

In [27]:
r

PyObject <module 'ROOT' from '/cvmfs/gm2.opensciencegrid.org/prod8/external/root/v6_08_06g/Linux64bit+2.6-2.12-e14-prof/lib/ROOT.pyc'>

In [30]:
r[:art][:InputTag]

PyObject <class 'ROOT.art.InputTag'>